<a href="https://colab.research.google.com/github/domingues100/IEEE---Water_Level/blob/main/OD_RETINANET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inicialization**

Install detectron2 and clone cocosplit

-Restart environment after that

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!git clone https://github.com/akarazniewicz/cocosplit.git
!pip install funcy
!pip install scikit-multilearn

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ahs45o2s
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ahs45o2s
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0ae803b1449cd2d3f8fa1b7c0f59356db10b3083
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 773.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6147884 sha256=ff9f413bab77db3d8798e85a978

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TRAIN MODEL

***Libraries***

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from google.colab.patches import cv2_imshow
import cv2
import os
import shutil

***Define some augmentations***

In [ ]:
class CustomAugmentation(T.Augmentation):
    def get_transform(self, image):
        transforms = [
            T.RandomFlip(prob=0.5, horizontal=True),
            T.RandomBrightness(0.1, 0.5),
            T.RandomContrast(0.1, 0.5),
            T.RandomRotation(angle=[-25, 25]),
            T.RandomNoise(intensity=0.12),
            T.RandomZoom(scale=(0.1, 0.3)),
            T.ElasticTransform(alpha=1.2, sigma=0.2),
            T.CropAndPad(percent=(0, 0.4))
        ]
        return T.TransformList(transforms)

***Split Coco***

I started with RCNN, so i used the same jsons for retinanet

In [ ]:
#thats the same name that i used before, just to load the same archives
teste = "teste_fasterrcnn"
test_json = f"/content/drive/MyDrive/Transaciones/{test}/test.json"
train_json = f"/content/drive/MyDrive/Transaciones/{test}/train.json"
images = "/content/drive/MyDrive/IC_Machine_Learning/images"

#that's the new name:
teste = 'test_Retinanet'

register_coco_instances(f"test_dataset_no_crop_{teste}", {}, test_json, images)
register_coco_instances(f"train_dataset_no_crop_{teste}", {}, train_json, images)

DatasetCatalog.register("test_dataset", lambda: register_coco_instances(f"test_dataset_no_crop_{test}", {}, test_json, images), )
DatasetCatalog.register("train_dataset", lambda: register_coco_instances(f"train_dataset_no_crop_{test}", {}, train_json, images))

test_dataset_no_crop_metadata = MetadataCatalog.get(f"test_dataset_no_crop_{test}")
train_dataset_no_crop_metadata = MetadataCatalog.get(f"train_dataset_no_crop_{test}")

In [ ]:
import gc
gc.collect()

57

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (f"train_dataset_no_crop_{test}",)
cfg.DATASETS.TEST = (f"teste_dataset_no_crop_{test}",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.RETINANET.NUM_CLASSES = 8
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 384
cfg.INPUT.MIN_SIZE_TEST = 384
cfg.INPUT.MAX_SIZE_TEST = 384

cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

OUTPUT_DIR = f"/content/drive/MyDrive/Transaciones/{teste}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
cfg.OUTPUT_DIR = OUTPUT_DIR

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[05/16 13:09:04 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

model_final_971ab9.pkl: 228MB [00:01, 150MB/s]                           
head.cls_score.{bias, weight}
  pixel_mean
  pixel_std


[05/16 13:09:07 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/16 13:13:01 d2.utils.events]:  eta: 0:38:02  iter: 19  total_loss: 1.722  loss_cls: 1.375  loss_box_reg: 0.3865    time: 10.8445  last_time: 9.5062  data_time: 10.2593  last_data_time: 8.8258   lr: 7.6924e-05  max_mem: 2626M
[05/16 13:16:04 d2.utils.events]:  eta: 0:32:56  iter: 39  total_loss: 0.973  loss_cls: 0.8461  loss_box_reg: 0.138    time: 9.8403  last_time: 8.9666  data_time: 8.2405  last_data_time: 8.2303   lr: 0.00015684  max_mem: 2626M
[05/16 13:18:59 d2.utils.events]:  eta: 0:29:42  iter: 59  total_loss: 0.5407  loss_cls: 0.4328  loss_box_reg: 0.0981    time: 9.4629  last_time: 9.6552  data_time: 8.0558  last_data_time: 8.9592   lr: 0.00023676  max_mem: 2626M
[05/16 13:21:54 d2.utils.events]:  eta: 0:26:26  iter: 79  total_loss: 0.3712  loss_cls: 0.2947  loss_box_reg: 0.0814    time: 9.2845  last_time: 9.0531  data_time: 8.0734  last_data_time: 8.3854   lr: 0.00031668  max_mem: 2626M
[05/16 13:24:51 d2.utils.events]:  eta: 0:22:55  iter: 99  total_loss: 0.3057  loss_cl

***Evaluate the perfomance***

In [ ]:
evaluator = COCOEvaluator(f"test_dataset_no_crop_{test}", cfg, False, output_dir=f"/content/drive/MyDrive/Transaciones/{test}")
val_loader = build_detection_test_loader(cfg, f"test_dataset_no_crop_{test}")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

#Load Model and test/calculate IoU


In [ ]:
teste = "teste_retinanet"

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml")) #FOR RETINANET
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = f"/content/drive/MyDrive/Transaciones/{teste}/model_final.pth"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.RETINANET.NUM_CLASSES = 8
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 384
cfg.INPUT.MIN_SIZE_TEST = 384
cfg.INPUT.MAX_SIZE_TEST = 384
cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

model = DefaultPredictor(cfg).model
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)


{'trainer': {'iteration': 249,
  'hooks': {'LRScheduler': {'base_lrs': [0.001], 'last_epoch': 250}},
  '_trainer': {'iteration': 249,
   'optimizer': {'state': {0: {'momentum_buffer': tensor([[[[-3.2732e-04]],
      
               [[-4.9053e-04]],
      
               [[-1.4050e-04]],
      
               ...,
      
               [[-5.7662e-04]],
      
               [[-4.9570e-04]],
      
               [[-2.8771e-04]]],
      
      
              [[[-1.2469e-04]],
      
               [[-9.1270e-06]],
      
               [[ 8.9718e-04]],
      
               ...,
      
               [[-2.3145e-04]],
      
               [[-9.8286e-04]],
      
               [[-7.2103e-04]]],
      
      
              [[[-2.3696e-04]],
      
               [[-1.6640e-03]],
      
               [[-1.8300e-03]],
      
               ...,
      
               [[ 1.3000e-05]],
      
               [[-2.2661e-04]],
      
               [[-7.6599e-04]]],
      
      
              .

In [ ]:
#test_json original
test_json = "/content/drive/MyDrive/Transaciones/test_fasterrcnn/train.json"
images = "/content/drive/MyDrive/IC_Machine_Learning/images"

In [ ]:
register_coco_instances("test_dataset_retina", {}, teste_json, images_originais)

DatasetCatalog.register("test_dataset", lambda: register_coco_instances("test_dataset_retina", {}, test_json, images_originais), )

teste_dataset_no_crop_metadata = MetadataCatalog.get("test_dataset_retina")

***Para usar o evaluator***

In [ ]:
evaluator = COCOEvaluator("test_dataset_retina", cfg, False, output_dir=f"/content/drive/MyDrive/IC_Machine_Learning/Deteccao_detectron2/{teste}")
val_loader = build_detection_test_loader(cfg, "test_dataset_retina")
print(inference_on_dataset(model, val_loader, evaluator))

***Fazer as predições***

***Predictions and Crop BBOX***

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)
best_predictions = []
gt_bbox = []
names = []
teste_dataset_dicts = load_coco_json(teste_json, images_originais)

output_images_dir = f"/content/drive/MyDrive/Transaciones/Images_RCNN"
os.makedirs(output_images_dir, exist_ok=True)

for d in teste_dataset_dicts:
    im = cv2.imread(d["file_name"])

    #saving gt bbox
    gt_bbox.append(d["annotations"][0]["bbox"])
    names.append(d["file_name"])
    outputs = predictor(im)

    detections = outputs["instances"].to("cpu")

    if len(detections) > 0:
        scores = detections.scores.tolist()
        max_score_idx = scores.index(max(scores))
        best_detection = detections[max_score_idx]
        bbox = best_detection.pred_boxes.tensor.cpu().numpy()[0]

        #saving best_prediction bbox
        best_predictions.append(bbox)
        x, y, w, h = bbox
        x, y, w, h = int(x), int(y), int(w), int(h)

        cropped_image = im[y:h, x:w]

        # Salvar o recorte
        output_image_path = os.path.join(output_images_dir, os.path.basename(d["file_name"]))
        cv2.imwrite(output_image_path, cropped_image)


***Código para calcular o IoU***

In [ ]:
def gt_bbox_coco_format(gt_bbox):
  gt_bbox_copy = gt_bbox.copy()
  for item in gt_bbox_copy:
    x, y, w, h = item
    item[2] = x+w
    item[3] = y+h
  return gt_bbox_copy

def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - box1: Tuple (x1, y1, x2, y2) representing the coordinates of the first bounding box.
    - box2: Tuple (x1, y1, x2, y2) representing the coordinates of the second bounding box.

    Returns:
    - IoU: Intersection over Union value.
    """

    # Calculate the intersection coordinates
    x1_i = max(box1[0], box2[0])
    y1_i = max(box1[1], box2[1])
    x2_i = min(box1[2], box2[2])
    y2_i = min(box1[3], box2[3])

    # Calculate the intersection area
    intersection_area = max(0, x2_i - x1_i + 1) * max(0, y2_i - y1_i + 1)

    # Calculate the union area
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_box2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = area_box1 + area_box2 - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    # append iou in a list
    iou_list.append(iou)


#list to store iou
iou_list = []

formated_gt_bbox = gt_bbox_coco_format(gt_bbox)

for item1, item2 in zip(formated_gt_bbox, best_predictions):
  calculate_iou(item1, item2)

mean_iou = sum(iou_list)/len(iou_list)
print(mean_iou)
